# Neptune + Optuna

## Before you start

### Install dependencies

In [1]:
! pip install --quiet optuna==2.3.0 neptune-client==0.4.125 neptune-contrib['monitoring']==0.24.8

### Import libraries

In [3]:
import lightgbm as lgb
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

### Create a sample `objective` function for Optuna

In [4]:
def objective(trial):
   
   data, target = load_breast_cancer(return_X_y=True)
   train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
   dtrain = lgb.Dataset(train_x, label=train_y)

   param = {
      'objective': 'binary',
      'metric': 'binary_logloss',
      'num_leaves': trial.suggest_int('num_leaves', 2, 256),
      'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
      'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
      'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
   }

   gbm = lgb.train(param, dtrain)
   preds = gbm.predict(test_x)
   accuracy = roc_auc_score(test_y, preds)
   
   return accuracy

### Initialize Neptune

In [5]:
import neptune

In [6]:
neptune.init(api_token='ANONYMOUS', project_qualified_name='shared/optuna-integration')

Project(shared/optuna-integration)

## Quickstart

### Step 1: Create an Experiment

In [7]:
neptune.create_experiment('optuna-sweep')

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/shared/optuna-integration/e/OP-8


Experiment(OP-8)

### Step 2: Create the Neptune Callback

In [8]:
import neptunecontrib.monitoring.optuna as opt_utils

In [9]:
neptune_callback = opt_utils.NeptuneCallback()

### Step 3: Run Optuna with the Neptune Callback

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, callbacks=[neptune_callback])

[I 2020-11-19 11:44:22,644] A new study created in memory with name: no-name-3cbca82e-2292-4826-8bdc-d85c8e8de7da
[I 2020-11-19 11:44:22,683] Trial 0 finished with value: 0.9878406708595389 and parameters: {'num_leaves': 223, 'feature_fraction': 0.7660474749933167, 'bagging_fraction': 0.4276766969511315, 'min_child_samples': 60}. Best is trial 0 with value: 0.9878406708595389.
[I 2020-11-19 11:44:23,238] Trial 1 finished with value: 0.9821057012068248 and parameters: {'num_leaves': 99, 'feature_fraction': 0.585439209659321, 'bagging_fraction': 0.9291986066766992, 'min_child_samples': 52}. Best is trial 0 with value: 0.9878406708595389.
[I 2020-11-19 11:44:23,266] Trial 2 finished with value: 0.9985632183908045 and parameters: {'num_leaves': 7, 'feature_fraction': 0.8866600442201023, 'bagging_fraction': 0.8108011257539387, 'min_child_samples': 26}. Best is trial 2 with value: 0.9985632183908045.
[I 2020-11-19 11:44:23,293] Trial 3 finished with value: 0.9977111943404078 and parameters: 

### Step 4: Stop logging

In [17]:
neptune.stop()

## Advanced Options

### Log charts and study object during sweep

### Log charts and study object after the sweep

## Explore results in the Neptune UI